In [1]:
 
import pandas as pd 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 

from wordcloud import wordcloud 
import nltk 
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adity\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
df=pd.read_csv("spam_ham_dataset.csv")
df.head()

,label,text,transformed_text
0,0,Subject: enron methanol ; meter # : 988291\r\n...,subject enron methanol meter 988291 follow not...
1,0,"Subject: hpl nom for january 9 , 2001\r\n( see...",subject hpl nom januari 9 2001 see attach file...
2,0,"Subject: neon retreat\r\nho ho ho , we ' re ar...",subject neon retreat ho ho ho around wonder ti...
3,1,"Subject: photoshop , windows , office . cheap ...",subject photoshop window offic cheap main tren...
4,0,Subject: re : indian springs\r\nthis deal is t...,subject indian spring deal book teco pvr reven...


In [5]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['label'] = le.fit_transform(df['label'])
df.head()

,label,text,transformed_text
0,0,Subject: enron methanol ; meter # : 988291\r\n...,subject enron methanol meter 988291 follow not...
1,0,"Subject: hpl nom for january 9 , 2001\r\n( see...",subject hpl nom januari 9 2001 see attach file...
2,0,"Subject: neon retreat\r\nho ho ho , we ' re ar...",subject neon retreat ho ho ho around wonder ti...
3,1,"Subject: photoshop , windows , office . cheap ...",subject photoshop window offic cheap main tren...
4,0,Subject: re : indian springs\r\nthis deal is t...,subject indian spring deal book teco pvr reven...


In [7]:
from nltk.stem.porter import PorterStemmer 
import string 

ps=PorterStemmer()


In [8]:
def tranform_text(text):
    text=text.lower()
    text=nltk.word_tokenize(text)

    y=[]
    for i in text:
        if i.isalnum():
            y.append(i)

    text=y[:]
    y.clear()

    for i in text:
        if i not in stopwords.words('english') and i not in string.punctuation:
            y.append(i)

    text=y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i))

    return " ".join(y)

In [9]:
tranform_text("This is a sample text, with some punctuation and stopwords!")


'sampl text punctuat stopword'

In [10]:
df['transformed_text'] = df['text'].apply(tranform_text)
df.head()

,label,text,transformed_text
0,0,Subject: enron methanol ; meter # : 988291\r\n...,subject enron methanol meter 988291 follow not...
1,0,"Subject: hpl nom for january 9 , 2001\r\n( see...",subject hpl nom januari 9 2001 see attach file...
2,0,"Subject: neon retreat\r\nho ho ho , we ' re ar...",subject neon retreat ho ho ho around wonder ti...
3,1,"Subject: photoshop , windows , office . cheap ...",subject photoshop window offic cheap main tren...
4,0,Subject: re : indian springs\r\nthis deal is t...,subject indian spring deal book teco pvr reven...


In [11]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer 
tfidf=TfidfVectorizer(max_features=500,stop_words='english')


In [12]:
X=tfidf.fit_transform(df['transformed_text']).toarray()
y=df['label'].values 


In [13]:
from sklearn.model_selection import train_test_split 
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [14]:
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC 
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.ensemble import GradientBoostingClassifier 
from xgboost import XGBClassifier 


In [15]:
svc=SVC(kernel='sigmoid',C=1,gamma=1.0)
knc=KNeighborsClassifier(n_neighbors=5,metric='minkowski',p=2)
lr=LogisticRegression(C=1,solver='liblinear')   
mnb=MultinomialNB(alpha=1.0)
dtc=DecisionTreeClassifier(criterion='gini',max_depth=5)
rfc=RandomForestClassifier(n_estimators=100,criterion='gini',max_depth=5)
gbc=GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=3)
xgbc=XGBClassifier(n_estimators=100,learning_rate=0.1,max_depth=3)
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report

In [16]:
clfs= {
    'SVC': svc,
    'KNeighborsClassifier': knc,
    'LogisticRegression': lr,
    'MultinomialNB': mnb,
    'DecisionTreeClassifier': dtc,
    'RandomForestClassifier': rfc,
    'GradientBoostingClassifier': gbc,
    'XGBClassifier': xgbc
}

In [17]:
def train_classifier(clf,x_train,y_train,x_test,y_test):
    clf.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    
    print("Classifier:",clf.__class__.__name__)
    print("Accuracy:",accuracy_score(y_test,y_pred))
    print("Confusion Matrix:\n",confusion_matrix(y_test,y_pred))
    print("Classification Report:\n",classification_report(y_test,y_pred))
    print("\n")

In [18]:
train_classifier(svc,x_train,y_train,x_test,y_test)
train_classifier(knc,x_train,y_train,x_test,y_test)
train_classifier(lr,x_train,y_train,x_test,y_test)
train_classifier(mnb,x_train,y_train,x_test,y_test)
train_classifier(dtc,x_train,y_train,x_test,y_test)
train_classifier(rfc,x_train,y_train,x_test,y_test)
train_classifier(gbc,x_train,y_train,x_test,y_test)
train_classifier(xgbc,x_train,y_train,x_test,y_test)


Classifier: SVC
Accuracy: 0.93
Confusion Matrix:
 [[69  6]
 [ 1 24]]
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.92      0.95        75
           1       0.80      0.96      0.87        25

    accuracy                           0.93       100
   macro avg       0.89      0.94      0.91       100
weighted avg       0.94      0.93      0.93       100



Classifier: KNeighborsClassifier
Accuracy: 0.92
Confusion Matrix:
 [[70  5]
 [ 3 22]]
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.93      0.95        75
           1       0.81      0.88      0.85        25

    accuracy                           0.92       100
   macro avg       0.89      0.91      0.90       100
weighted avg       0.92      0.92      0.92       100



Classifier: LogisticRegression
Accuracy: 0.94
Confusion Matrix:
 [[70  5]
 [ 1 24]]
Classification Report:
               precision    reca